### Set API Key and Load LLM

In [5]:
import os
os.environ["OPENAI_API_KEY"] = "voc-00000000000000000000000000000000abcd.12345678"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

from langchain.llms import OpenAI

### Import LangChain Modules

In [6]:
import os
from langchain.llms import OpenAI
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate


### Define Listing Generator using Chat Model

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

chat_llm = ChatOpenAI(temperature=0.7)

def generate_listing_chat():
    prompt = "Generate a realistic real estate listing including: neighborhood, price, bedrooms, bathrooms, house size, description, and neighborhood description."
    try:
        response = chat_llm([HumanMessage(content=prompt)])
        return response.content
    except Exception as e:
        print("Error occurred:", e)
        return None

# Test with one listing
print(generate_listing_chat())


Neighborhood: Silver Lake, Los Angeles, CA
Price: $1,200,000
Bedrooms: 4
Bathrooms: 3
House Size: 2,500 sqft

Description: Welcome to this stunning modern home in the highly desirable Silver Lake neighborhood. This 4 bedroom, 3 bathroom home features an open concept floor plan with high ceilings, hardwood floors, and plenty of natural light. The gourmet kitchen is a chef's dream with stainless steel appliances, quartz countertops, and a large island. The spacious master suite boasts a walk-in closet and a luxurious en-suite bathroom with a soaking tub and separate shower. Enjoy the California sunshine in the private backyard with a patio and lush landscaping.

Neighborhood Description: Silver Lake is known for its trendy boutiques, eclectic cafes, and vibrant arts scene. Residents enjoy easy access to the Silver Lake Reservoir, where you can go for a jog or enjoy a leisurely walk with stunning views of the city. The neighborhood is also home to some of the best restaurants in Los Angel

### Generate 10 Synthetic Real Estate Listings

In [10]:
listings = []

for i in range(10):
    print(f"Generating listing {i+1}...")
    listing = generate_listing_chat()
    if listing:
        listings.append(listing)

# Print a sample listing
print("\nSample Listing:\n")
print(listings[0])


Generating listing 1...
Generating listing 2...
Generating listing 3...
Generating listing 4...
Generating listing 5...
Generating listing 6...
Generating listing 7...
Generating listing 8...
Generating listing 9...
Generating listing 10...

Sample Listing:

Welcome to this charming 3-bedroom, 2-bathroom home located in the desirable Oak Park neighborhood. Priced at $350,000, this 1,800 square foot house features a spacious living room, updated kitchen with stainless steel appliances, and a cozy backyard perfect for entertaining.

The Oak Park neighborhood is known for its tree-lined streets, friendly community atmosphere, and proximity to parks, schools, and shopping centers. With easy access to public transportation and major highways, this location is ideal for commuters and families alike.

Don't miss out on the opportunity to own this lovely home in Oak Park - schedule a showing today!


### Save Listings to File

In [11]:
# Save to file for Step 8 submission
with open("listings.txt", "w") as f:
    for listing in listings:
        f.write(listing + "\n\n")


In [12]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document


### Initialize Embedding and ChromaDB

In [13]:
# Initialize embedding function
embedding_fn = OpenAIEmbeddings()

# Initialize vector store
vectorstore = Chroma(collection_name="real_estate_listings", embedding_function=embedding_fn)

# Convert listings to documents
docs = [Document(page_content=listing) for listing in listings]

# Add documents to vector database
vectorstore.add_documents(docs)

print("✅ All listings added to ChromaDB successfully!")


✅ All listings added to ChromaDB successfully!


### Perform a Test Semantic Search

In [14]:
query = "I want a 3-bedroom eco-friendly home with a garden in a quiet neighborhood."
results = vectorstore.similarity_search(query, k=3)

for i, res in enumerate(results):
    print(f"\nResult {i+1}:\n{res.page_content}\n")



Result 1:
Welcome to this charming 3-bedroom, 2-bathroom home located in the sought-after neighborhood of Willow Creek. Priced at $350,000, this spacious 1,800 square foot house features a cozy living room with a fireplace, a fully renovated kitchen with stainless steel appliances, and a large backyard perfect for entertaining.

The master bedroom boasts a walk-in closet and en-suite bathroom, while the two additional bedrooms are perfect for guests or a home office. The neighborhood of Willow Creek is known for its tree-lined streets, friendly neighbors, and top-rated schools. With easy access to shopping, dining, and parks, this home is perfect for families or anyone looking for a peaceful retreat in a vibrant community. Don't miss out on the opportunity to make this house your forever home!


Result 2:
Welcome to this charming 3-bedroom, 2-bathroom home located in the coveted neighborhood of Willow Creek. Priced at $350,000, this 1,800 square foot home offers a spacious layout with

### Collect Buyer Preferences

In [15]:
# Define questions and hard-coded answers
questions = [   
    "How big do you want your house to be?", 
    "What are 3 most important things for you in choosing this property?", 
    "Which amenities would you like?", 
    "Which transportation options are important to you?",
    "How urban do you want your neighborhood to be?"
]

answers = [
    "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
    "A quiet neighborhood, good local schools, and convenient shopping options.",
    "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."
]

# Combine into a full preference text
buyer_preferences = "\n".join([f"{q} {a}" for q, a in zip(questions, answers)])

print("Buyer Preferences:\n")
print(buyer_preferences)


Buyer Preferences:

How big do you want your house to be? A comfortable three-bedroom house with a spacious kitchen and a cozy living room.
What are 3 most important things for you in choosing this property? A quiet neighborhood, good local schools, and convenient shopping options.
Which amenities would you like? A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.
Which transportation options are important to you? Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.
How urban do you want your neighborhood to be? A balance between suburban tranquility and access to urban amenities like restaurants and theaters.


### Semantic Search Based on Preferences

In [16]:
# Perform semantic search
matched_listings = vectorstore.similarity_search(buyer_preferences, k=3)

# Preview results
for i, match in enumerate(matched_listings):
    print(f"\nMatched Listing {i+1}:\n{match.page_content}")



Matched Listing 1:
Welcome to this charming 3-bedroom, 2-bathroom home located in the desirable Oak Park neighborhood. Priced at $350,000, this 1,800 square foot house features a spacious living room, updated kitchen with stainless steel appliances, and a cozy backyard perfect for entertaining.

The Oak Park neighborhood is known for its tree-lined streets, friendly community atmosphere, and proximity to parks, schools, and shopping centers. With easy access to public transportation and major highways, this location is ideal for commuters and families alike.

Don't miss out on the opportunity to own this lovely home in Oak Park - schedule a showing today!

Matched Listing 2:
Welcome to this charming 3-bedroom, 2-bathroom home located in the sought-after neighborhood of Willow Creek. Priced at $350,000, this spacious 1,800 square foot house features a cozy living room with a fireplace, a fully renovated kitchen with stainless steel appliances, and a large backyard perfect for entertain

### Define LLM-based Personalization

In [17]:
def personalize_listing(original_description, preferences):
    prompt = f"""
You are a helpful real estate agent.

Given the following property listing:
{original_description}

And the buyer preferences:
{preferences}

Rewrite the listing to highlight the features that match the preferences.
Keep all factual details unchanged, just emphasize the relevant parts in an engaging way.
"""
    response = chat_llm([HumanMessage(content=prompt)])
    return response.content


### Generate Personalized Listings

In [18]:
for i, match in enumerate(matched_listings):
    print(f"\n🎯 Personalized Listing {i+1}:\n")
    print(personalize_listing(match.page_content, buyer_preferences))



🎯 Personalized Listing 1:

Welcome to this charming 3-bedroom, 2-bathroom home nestled in the serene Oak Park neighborhood. Priced at $350,000, this 1,800 square foot house boasts a spacious kitchen with stainless steel appliances, perfect for cooking up delicious meals. The cozy living room is ideal for relaxing with loved ones after a long day.

Oak Park is renowned for its peaceful surroundings, top-notch schools, and convenient shopping options just a stone's throw away. Enjoy the tranquility of tree-lined streets while still having easy access to public transportation and major highways for seamless commuting.

Step outside to the inviting backyard, perfect for gardening enthusiasts, and take advantage of the two-car garage for convenient parking. Stay warm and energy-efficient with the modern heating system in place.

Experience the perfect blend of suburban charm and urban convenience in Oak Park. Don't miss out on this opportunity - schedule a showing today and make this lovel

### Save Listings and Results

In [19]:
with open("listings.txt", "w") as f:
    for listing in listings:
        f.write(listing.strip() + "\n\n")


### Save Example Output for Submission

In [20]:
# Save example results to example_output.txt
with open("example_output.txt", "w") as f:
    f.write("=== BUYER PREFERENCES ===\n")
    f.write(buyer_preferences + "\n\n")
    
    f.write("=== MATCHED LISTING ===\n")
    f.write(matched_listings[0].page_content.strip() + "\n\n")
    
    personalized = personalize_listing(matched_listings[0].page_content, buyer_preferences)
    f.write("=== PERSONALIZED LISTING ===\n")
    f.write(personalized.strip())
